In [1]:
import pandas as pd
import pyodbc

In [2]:
# Connect to DataBank
connection_string = (
    "DSN=DataBank;"
    "Trusted_Connection=yes;"
)

con = pyodbc.connect(connection_string)

In [3]:
# Write sql query
qry = "SELECT * FROM ukri_ssd_snapshot.2024_10_04_ssd_outcome_intellectual_property;"

# Read sql query
df= pd.read_sql(qry, con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_4604\1573845778.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df= pd.read_sql(qry, con)


In [4]:
df['protection'].unique()

array(['Patent granted', 'Protection not required',
       'Copyrighted (e.g. software)', 'Patent application published',
       'Trade Mark', 'Protection Not Required',
       'Patent / Patent application'], dtype=object)

In [5]:
df['licensed'].unique()

array(['No', 'Yes', 'Commercial In Confidence', None], dtype=object)

In [6]:
df['application_reference'].replace("Unknown", None, inplace=True)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_4604\2917795902.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['application_reference'].replace("Unknown", None, inplace=True)


In [7]:
df['application_reference'].isna().sum()/len(df)*100

# 14.9% have Null application reference

14.922737306843267

In [8]:
df.dropna(subset='application_reference')['application_reference'].duplicated().sum()

3936

In [9]:
df_application = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2024_10_04_ssd_application;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_4604\3848588355.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_application = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2024_10_04_ssd_application;", con)


In [10]:
df_application['application_reference'].duplicated().sum()

7230

In [11]:
data_ = df_application.drop_duplicates(subset='application_reference', keep=False)


In [12]:
data_.columns

data_[['application_reference', 'award_reference', 'lead_organisation_id', 'organisation_source']]

,application_reference,award_reference,lead_organisation_id,organisation_source
1,10000,None,469,TFS
2,10001,None,34,TFS
5,10011,MR/Z503976/1,16608,Siebel
6,10013,None,22,TFS
7,10019,None,23,TFS
...,...,...,...,...
575933,1184,None,10007799,Unknown
575939,1213,CCF ST:Innovation Catalysed & Enabled (ICE) Le...,10007796,Unknown
575940,1212,CCF ST:AI SuperConnector,10003270,Unknown
575941,1211,CCF ST:North West Cyber Security Connect for C...,10007768,Unknown


In [13]:
data_merge = pd.merge(df, data_[['application_reference', 'award_reference', 'lead_organisation_id', 'organisation_source']],
          on='application_reference', how='left', indicator=True)

In [14]:
data_both = data_merge[data_merge['_merge'] == 'both']


In [15]:
print(len(data_both)/len(df)*100)

83.42163355408388


In [16]:
ssd_organisation = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2024_10_04_ssd_organisation;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_4604\2732986568.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ssd_organisation = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2024_10_04_ssd_organisation;", con)


In [17]:
data_both.columns

Index(['application_reference', 'award_reference_x', 'funder', 'administrator',
       'outcome_base_id', 'outcome_first_submitted', 'outcome_last_submitted',
       'year_protection_granted', 'protection', 'patent_id', 'description',
       'licensed', 'impact', 'url', 'record_title', 'award_reference_y',
       'lead_organisation_id', 'organisation_source', '_merge'],
      dtype='object')

In [18]:
ssd_organisation.columns

Index(['organisation_id', 'ukprn', 'organisation_city', 'organisation_country',
       'organisation_name', 'organisation_postcode', 'organisation_type',
       'organisation_status', 'itl1_id', 'itl1_name', 'itl2_id', 'itl2_name',
       'itl3_id', 'itl3_name', 'constituency_id', 'constituency_name',
       'organisation_source'],
      dtype='object')

In [27]:
data_both.drop(columns='_merge', inplace=True)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_4604\2809252394.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_both.drop(columns='_merge', inplace=True)


In [28]:
data_w_org = pd.merge(data_both, ssd_organisation, left_on=['lead_organisation_id', 'organisation_source'], right_on=['organisation_id', 'organisation_source'], 
                      how='left', indicator=True)

In [30]:
data_w_org[data_w_org['_merge'] == 'left_only']

,application_reference,award_reference_x,funder,administrator,outcome_base_id,outcome_first_submitted,outcome_last_submitted,year_protection_granted,protection,patent_id,...,organisation_status,itl1_id,itl1_name,itl2_id,itl2_name,itl3_id,itl3_name,constituency_id,constituency_name,_merge


In [31]:
print(len(data_both))
print(len(data_w_org))

7558
7558


In [32]:
len(data_w_org)/len(df)*100

83.42163355408388

In [34]:
print(len(data_w_org))

7558


In [35]:
print(len(df))

9060


In [38]:
print(df['application_reference'].isna().sum())

1352
